In [1]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 17: Set and Forget ---</h2><p>An early warning system detects an incoming <a href="https://en.wikipedia.org/wiki/Solar_flare">solar flare</a> and automatically activates the ship's electromagnetic shield. Unfortunately, this has cut off the Wi-Fi for many small robots that, unaware of the impending danger, are now trapped on exterior scaffolding on the unsafe side of the shield. To rescue them, you'll have to act quickly!</p>
<p>The only tools at your disposal are some wired cameras and a small vacuum robot currently asleep at its charging station. The video quality is poor, but the vacuum robot has a needlessly bright LED that makes it easy to spot no matter where it is.</p>
<p>An <a href="9">Intcode</a> program, the <em>Aft Scaffolding Control and Information Interface</em> (ASCII, your puzzle input), provides access to the cameras and the vacuum robot.  Currently, because the vacuum robot is asleep, you can only access the cameras.</p>
<p>Running the ASCII program on your Intcode computer will provide the current view of the scaffolds.  This is output, <span title="PURELY COINCIDENTALLY">purely coincidentally</span>, as <a href="https://simple.wikipedia.org/wiki/ASCII">ASCII code</a>: <code>35</code> means <code>#</code>, <code>46</code> means <code>.</code>, <code>10</code> starts a <a href="https://en.wikipedia.org/wiki/Newline#In_programming_languages">new line</a> of output below the current one, and so on. (Within a line, characters are drawn left-to-right.)</p>
<p>In the camera output, <code>#</code> represents a scaffold and <code>.</code> represents open space. The vacuum robot is visible as <code>^</code>, <code>v</code>, <code>&lt;</code>, or <code>&gt;</code> depending on whether it is facing up, down, left, or right respectively. When drawn like this, the vacuum robot is <em>always on a scaffold</em>; if the vacuum robot ever walks off of a scaffold and begins <em>tumbling through space uncontrollably</em>, it will instead be visible as <code>X</code>.</p>
<p>In general, the scaffold forms a path, but it sometimes loops back onto itself.  For example, suppose you can see the following view from the cameras:</p>
<pre><code>..#..........
..#..........
#######...###
#.#...#...#.#
#############
..#...#...#..
..#####...^..
</code></pre>
<p>Here, the vacuum robot, <code>^</code> is facing up and sitting at one end of the scaffold near the bottom-right of the image. The scaffold continues up, loops across itself several times, and ends at the top-left of the image.</p>
<p>The first step is to calibrate the cameras by getting the <em>alignment parameters</em> of some well-defined points.  Locate all <em>scaffold intersections</em>; for each, its alignment parameter is the distance between its left edge and the left edge of the view multiplied by the distance between its top edge and the top edge of the view.  Here, the intersections from the above image are marked <code>O</code>:</p>
<pre><code>..#..........
..#..........
##O####...###
#.#...#...#.#
##O###O###O##
..#...#...#..
..#####...^..
</code></pre>
<p>For these intersections:</p>
<ul>
<li>The top-left intersection is <code>2</code> units from the left of the image and <code>2</code> units from the top of the image, so its alignment parameter is <code>2 * 2 = <em>4</em></code>.</li>
<li>The bottom-left intersection is <code>2</code> units from the left and <code>4</code> units from the top, so its alignment parameter is <code>2 * 4 = <em>8</em></code>.</li>
<li>The bottom-middle intersection is <code>6</code> from the left and <code>4</code> from the top, so its alignment parameter is <code><em>24</em></code>.</li>
<li>The bottom-right intersection's alignment parameter is <code><em>40</em></code>.</li>
</ul>
<p>To calibrate the cameras, you need the <em>sum of the alignment parameters</em>.  In the above example, this is <code><em>76</em></code>.</p>
<p>Run your ASCII program. <em>What is the sum of the alignment parameters</em> for the scaffold intersections?</p>
</article>


In [2]:
with open("../input/day17.txt") as f:
    puzzle = f.read()

In [3]:
from heapq import heappop, heappush
from itertools import takewhile
from math import prod
from re import finditer, split
from typing import Generator


from IntcodeComputer import IntcodeComputer


class Robot:

    def __init__(self, program: str) -> None:
        self.computer = IntcodeComputer(program)
        self.map = self.make_map(self.computer.run())
        self.scaffolding_points = self.get_alignment_parameters()
        self.start, self.end, self.count = self.get_start_and_end()

    def get_start_and_end(self) -> tuple[tuple[int, int], tuple[int, int]]:
        rows, cols = len(self.map), len(self.map[0])
        start, end = None, None
        count = 0

        for row, col in product(range(rows), range(cols)):
            if self.map[row][col] == 35:
                count += 1
                neigbors = sum(
                    1
                    for dr, dc in ((-1, 0), (1, 0), (0, -1), (0, 1))
                    if 0 <= row + dr < rows
                    and 0 <= col + dc < cols
                    and self.map[row + dr][col + dc] in [35, 94, 118, 60, 62]
                )
                if neigbors == 1:
                    end = row, col
            elif self.map[row][col] in [94, 118, 60, 62]:
                count += 1
                start = row, col

        return start, end, count

    def make_map(self, iterator: Generator[int | None, int, None]) -> list[list[int]]:
        map = [[]]
        for ch in iterator:
            if ch == 10:
                map.append([])
            else:
                map[-1].append(ch)
        while not map[-1]:
            map.pop()
        return map

    def get_alignment_parameters(self) -> set[tuple[int, int]]:
        rows, cols = len(self.map), len(self.map[0])
        return {
            (row, col)
            for row, col in product(range(1, rows - 1), range(1, cols - 1))
            if (
                self.map[row][col]
                == self.map[row - 1][col]
                == self.map[row + 1][col]
                == self.map[row][col - 1]
                == self.map[row][col + 1]
                == 35
            )
        }

    def sum_alignment_parameters(self) -> int:
        return sum(prod(p) for p in self.get_alignment_parameters())

    @classmethod
    def compress_instructions(cls, instructions: list[chr | int]) -> list[chr | int]:
        compressed_instructions = []
        i, n = 0, len(instructions)

        while i < n:
            if isinstance(instructions[i], int):
                count = 0
                while i < n and isinstance(instructions[i], int):
                    count += 1
                    i += 1
                compressed_instructions.append(count)
            else:
                compressed_instructions.append(instructions[i])
                i += 1

        return compressed_instructions

    def walk_scaffolding(self) -> list[int | str]:
        dirs = {94: 0, 62: 1, 118: 2, 60: 3}  # ^, >, v, <
        dxdy = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # ^, >, v, <

        rows, cols = len(self.map), len(self.map[0])
        row_start, col_start = self.start
        row_end, col_end = self.end

        r, c, d, instructions = (
            row_start,
            col_start,
            dirs[self.map[row_start][col_start]],
            [],
        )

        while True:
            if r == row_end and c == col_end:
                return self.compress_instructions(instructions)

            if not (0 <= r < rows and 0 <= c < cols):
                continue

            if self.map[r][c] == 46:
                continue

            dr, dc = dxdy[d]
            # move 1
            if (
                0 <= r + dr < rows
                and 0 <= c + dc < cols
                and self.map[r + dr][c + dc] != 46
            ):
                r, c, d = r + dr, c + dc, d
                instructions.append(1)
                continue

            # turn right
            dr = (d + 1) % 4
            rdr, rdc = dxdy[dr]
            if (
                0 <= r + rdr < rows
                and 0 <= c + rdc < cols
                and self.map[r + rdr][c + rdc] != 46
            ):
                r, c, d = r, c, dr
                instructions.append("R")
                continue

            # turn left
            dl = (d - 1) % 4
            ldr, ldc = dxdy[dl]
            if (
                0 <= r + ldr < rows
                and 0 <= c + ldc < cols
                and self.map[r + ldr][c + ldc] != 46
            ):
                r, c, d = (
                    r,
                    c,
                    dl,
                )
                instructions.append("L")

    def to_list(self, s: str) -> list[chr | int]:
        return [
            int(ch) if ch.isdecimal() else ch
            for p in zip(
                (ch for ch in split(r"\d+", s) if ch),
                (ch for ch in split(r"\D+", s) if ch),
            )
            for ch in p
        ]

    def find_functions(self, scaffold: str):
        scf = "".join(str(c) for c in scaffold)
        heap = [([], scf)]
        while heap:
            functions, s = heappop(heap)

            if not s and len(functions) == 3:
                a, b, c = functions
                return (
                    self.to_list(a),
                    self.to_list(b),
                    self.to_list(c),
                    self.convert(a, b, c, scf),
                )

            if len(functions) == 3:
                continue

            for m in takewhile(lambda m: m.end() < 20, finditer(r"[0-9]+", s)):
                fn = s[: m.end()]
                heappush(heap, (functions + [fn], s.replace(fn, "")))

    def convert(self, a: str, b: str, c: str, scf: str) -> list[int | str]:
        return list(scf.replace(a, "A").replace(b, "B").replace(c, "C"))

    def asciify(self, l: list[int | str]) -> list[int]:
        s = ",".join(str(ch) for ch in l)
        return [ord(ch) for ch in s]

    def collect_dust(self, do_print=False) -> int:
        scaffold = self.walk_scaffolding()
        a, b, c, sfc = map(self.asciify, self.find_functions(scaffold))
        prg = (sfc + [10], a + [10], b + [10], c + [10])

        self.computer.state[0] = 2
        iter = self.computer.run()
        for inp in prg:
            while iter.send(None) != 10:
                ...

            iter.send(None)

            for ch in inp:
                iter.send(ch)

        while iter.send(None) != 10:
            ...

        iter.send(ord("y"))
        iter.send(10)

        try:
            while True:
                collected = iter.send(10)
                if do_print:
                    if collected == 10:
                        print()
                    else:
                        print(chr(collected), end="")
        except StopIteration:
            return collected

    def print_map(self) -> str:
        plt.imshow(self.map, cmap=plt.cm.viridis)
        plt.show()

    def __repr__(self) -> str:
        return "\n".join(
            "".join(
                "O" if (row, col) in self.scaffolding_points else chr(ch)
                for col, ch in enumerate(l)
            )
            for row, l in enumerate(self.map)
        )


print(f"Part I: {Robot(puzzle).sum_alignment_parameters()}")

Part I: 4112


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4112</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now for the tricky part: notifying all the other robots about the solar flare.  The vacuum robot can do this automatically if it gets into range of a robot. However, you can't see the other robots on the camera, so you need to be thorough instead: you need to make the vacuum robot <em>visit every part of the scaffold at least once</em>.</p>
<p>The vacuum robot normally wanders randomly, but there isn't time for that today.  Instead, you can <em>override its movement logic</em> with new rules.</p>
<p>Force the vacuum robot to wake up by changing the value in your ASCII program at address <code>0</code> from <code>1</code> to <code><em>2</em></code>. When you do this, you will be automatically prompted for the new movement rules that the vacuum robot should use. The ASCII program will use input instructions to receive them, but they need to be provided as ASCII code; end each line of logic with a single newline, ASCII code <code>10</code>.</p>
<p>First, you will be prompted for the <em>main movement routine</em>.  The main routine may only call the <em>movement functions</em>: <code>A</code>, <code>B</code>, or <code>C</code>. Supply the movement functions to use as ASCII text, separating them with commas (<code>,</code>, ASCII code <code>44</code>), and ending the list with a newline (ASCII code <code>10</code>). For example, to call <code>A</code> twice, then alternate between <code>B</code> and <code>C</code> three times, provide the string <code>A,A,B,C,B,C,B,C</code> and then a newline.</p>
<p>Then, you will be prompted for each <em>movement function</em>. Movement functions may use <code>L</code> to <em>turn left</em>, <code>R</code> to <em>turn right</em>, or a number to <em>move forward</em> that many units.  Movement functions may not call other movement functions.  Again, separate the actions with commas and end the list with a newline.  For example, to move forward <code>10</code> units, turn left, move forward <code>8</code> units, turn right, and finally move forward <code>6</code> units, provide the string <code>10,L,8,R,6</code> and then a newline.</p>
<p>Finally, you will be asked whether you want to see a <em>continuous video feed</em>; provide either <code>y</code> or <code>n</code> and a newline.  Enabling the continuous video feed can help you see what's going on, but it also requires a significant amount of processing power, and may even cause your Intcode computer to overheat.</p>
<p>Due to the limited amount of memory in the vacuum robot, the ASCII definitions of the main routine and the movement functions may each contain <em>at most 20 characters</em>, not counting the newline.</p>
<p>For example, consider the following camera feed:</p>
<pre><code>#######...#####
#.....#...#...#
#.....#...#...#
......#...#...#
......#...###.#
......#.....#.#
^########...#.#
......#.#...#.#
......#########
........#...#..
....#########..
....#...#......
....#...#......
....#...#......
....#####......
</code></pre>
<p>In order for the vacuum robot to <em>visit every part of the scaffold at least once</em>, one path it could take is:</p>
<pre><code>R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2</code></pre>
<p>Without the memory limit, you could just supply this whole string to function <code>A</code> and have the main routine call <code>A</code> once.  However, you'll need to split it into smaller parts.</p>
<p>One approach is:</p>
<ul>
<li><em>Main routine: <code>A,B,C,B,A,C</code></em><br>(ASCII input: <code>65</code>, <code>44</code>, <code>66</code>, <code>44</code>, <code>67</code>, <code>44</code>, <code>66</code>, <code>44</code>, <code>65</code>, <code>44</code>, <code>67</code>, <code>10</code>)</li>
<li><em>Function <code>A</code>:&nbsp;&nbsp;&nbsp;<code>R,8,R,8</code></em><br>(ASCII input: <code>82</code>, <code>44</code>, <code>56</code>, <code>44</code>, <code>82</code>, <code>44</code>, <code>56</code>, <code>10</code>)</li>
<li><em>Function <code>B</code>:&nbsp;&nbsp;&nbsp;<code>R,4,R,4,R,8</code></em><br>(ASCII input: <code>82</code>, <code>44</code>, <code>52</code>, <code>44</code>, <code>82</code>, <code>44</code>, <code>52</code>, <code>44</code>, <code>82</code>, <code>44</code>, <code>56</code>, <code>10</code>)</li>
<li><em>Function <code>C</code>:&nbsp;&nbsp;&nbsp;<code>L,6,L,2</code></em><br>(ASCII input: <code>76</code>, <code>44</code>, <code>54</code>, <code>44</code>, <code>76</code>, <code>44</code>, <code>50</code>, <code>10</code>)</li>
</ul>
<p>Visually, this would break the desired path into the following parts:</p>
<pre><code>A,        B,            C,        B,            A,        C
R,8,R,8,  R,4,R,4,R,8,  L,6,L,2,  R,4,R,4,R,8,  R,8,R,8,  L,6,L,2

CCCCCCA...BBBBB
C.....A...B...B
C.....A...B...B
......A...B...B
......A...CCC.B
......A.....C.B
^AAAAAAAA...C.B
......A.A...C.B
......AAAAAA#AB
........A...C..
....BBBB#BBBB..
....B...A......
....B...A......
....B...A......
....BBBBA......
</code></pre>

<p>Of course, the scaffolding outside your ship is much more complex.</p>
<p>As the vacuum robot finds other robots and notifies them of the impending solar flare, it also can't help but leave them squeaky clean, collecting any space dust it finds. Once it finishes the programmed set of movements, assuming it hasn't drifted off into space, the cleaning robot will return to its docking station and report the amount of space dust it collected as a large, non-ASCII value in a single output instruction.</p>
<p>After visiting every part of the scaffold at least once, <em>how much dust does the vacuum robot report it has collected?</em></p>
</article>

</main>


In [4]:
print(Robot(puzzle))
print(f"Part II: {Robot(puzzle).collect_dust(do_print=True)}")

#####........................
#...#........................
#...#........................
#...#........................
#...#####.........###########
#.......#.........#.........#
#.......#.........#.........#
#.......#.........#.........#
#.......#.........#.....#####
#.......#.........#.....#....
########O##.......#.....#....
........#.#.......#.....#....
..^#####O#O##.####O######....
........#.#.#.#...#..........
........##O#O#O####..........
..........#.#.#..............
........##O#O#O####..........
........#.#.#.#...#..........
......##O#O#O#O##.#..........
......#.#.#.#.#.#.#..........
######O#O##.#.#.#.#..........
#.....#.#...#.#.#.#..........
#.....##O####.##O##..........
#.......#.......#............
#.......#.......#............
........#.......#............
........########O##..........
................#.#..........
................##O##........
..................#.#........
..................#.#........
..................#.#........
..........#####...#.#........
..........

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>578918</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
